In [16]:
import pandas as pd
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from pathlib import Path
import warnings
import scipy.stats as stats 

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from app.assets import preprocessing

alt.data_transformers.enable("vegafusion")

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Suppress warnings
warnings.filterwarnings('ignore')

In [17]:
# Load processed data using the data loading function
data = preprocessing.load_processed_data()

df_customer = data['customer']
df_order = data['order']
df_geo = data['geo']
df_order_item = data['order_item']
df_product = data['product']
df_order_payment = data['order_payment']
df_order_review = data['order_review']
df_seller = data['seller']


## KPI's

In [18]:
# Total sales
def get_total_revenue(df_order_item: pd.DataFrame) -> str:
    return f"${df_order_item['price'].sum().astype(int):,.0f}"

get_total_revenue(df_order_item)


'$13,406,593'

In [19]:
def get_total_orders(df: pd.DataFrame) -> str:
    return f"{df['order_id'].nunique():,.0f}"

get_total_orders(df_order)

'95,133'

In [20]:
# Total customers

def get_total_customers(df: pd.DataFrame) -> str:
    return f"{df['customer_id'].nunique():,.0f}"

get_total_customers(df_customer)

'99,440'

In [21]:
# Highest selling city

def get_highest_selling_cities(data: dict[str, pd.DataFrame]) -> str:
    df_order = data['order']
    df_customer = data['customer']

    order_cols = ['order_id', 'customer_id']
    customer_cols = ['customer_id', 'city']

    highest_selling_cities = (df_customer[customer_cols]
                    .merge(df_order[order_cols], on='customer_id', how='inner')
                    .groupby('city').agg({'order_id': 'count'})
                    .sort_values(by='order_id', ascending=False))
    
    return highest_selling_cities


highest_selling_cities = get_highest_selling_cities(data)

highest_selling_cities.idxmax()


order_id    sao paulo
dtype: object

In [22]:
# # Highest selling category

def get_highest_selling_categories(data: dict[str, pd.DataFrame]) -> pd.DataFrame:
    df_order_item = data['order_item']

    highest_selling_categories = (df_order_item
        .groupby('category_name')
        .agg({'order_id': 'count'}).sort_values(by='order_id', ascending=False))

    return highest_selling_categories

highest_selling_categories = get_highest_selling_categories(data)

highest_selling_categories.idxmax()

order_id    bed_bath_table
dtype: object

## Sales Analysis

In [23]:
def get_sales_by_region_category(data: dict[str, pd.DataFrame]) -> pd.DataFrame:
    """
    Merge the customer, orders, geo, order item, to get sales by region
    Args:
        data: dict[str, pd.DataFrame]
            Data:
                - customer: pd.DataFrame
                - order: pd.DataFrame
                - geo: pd.DataFrame
                - order_item: pd.DataFrame
    Returns:
        pd.DataFrame - Columns: category_name, region, sales, order_count
    
    Notes:
        - The merging could be done during preprocessing to avoid redundant merging
    """
    
    df_customer = data['customer']
    df_orders = data['order']
    df_geo = data['geo']
    df_order_item = data['order_item']


    # Get unique zips with city, state, lat, lng, and region
    geo_cols = ['zip_code_prefix','region', 'city', 'state', 'latitude', 'longitude']
    unique_zips = (df_geo[geo_cols]
                    .groupby('zip_code_prefix')
                    .first())

    # Merge order and customer data to get zips
    customer_order = df_orders.merge(df_customer[['customer_id','zip_code_prefix']], on='customer_id', how='inner')

    # Merge zips and geo location with customer order data
    customer_order_geo = unique_zips.merge(customer_order, on='zip_code_prefix', how='inner')
    
    customer_order_geo_product = customer_order_geo.merge(df_order_item, on='order_id', how='inner')

    # Calculate sales by region and product category
    sales_by_region = (customer_order_geo_product
            .groupby(["category_name", "region"])
            .agg({"price": "sum", "order_id": "count"})
            .reset_index()
            .rename(columns={"price": "sales", "order_id": "order_count"})
            )
    
    return sales_by_region

sales_by_region_category = get_sales_by_region_category(data)


In [24]:
def calculate_ARPU(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the ARPU for each product category and region
    Args:
        df: pd.DataFrame - Requires sales and order_count columns
    Returns:
        pd.DataFrame - Columns: Product Category, Region, Sales, Orders, ARPU
    """


    df_with_ARPU = (df
            .assign(ARPU=lambda x: round(x["sales"] / x["order_count"], 2))
            .sort_values(by="ARPU", ascending=False))
    return df_with_ARPU

sales_by_region_category = calculate_ARPU(sales_by_region_category)

In [25]:
sales_by_region_category

,category_name,region,sales,order_count,ARPU
300,small_appliances_home_oven_and_coffee,North,3998.00,2,1999.00
68,computers,South,21293.85,17,1252.58
66,computers,North,10995.90,10,1099.59
69,computers,Southeast,135959.64,132,1030.00
65,computers,Central-West,23753.89,24,989.75
...,...,...,...,...,...
219,home_comfort_2,Southeast,740.58,36,20.57
120,dvds_blu_ray,Northeast,101.60,5,20.32
259,music,North,16.10,1,16.10
216,home_comfort_2,Central-West,90.30,7,12.90


In [26]:
# ARPU and total sales by Region and Product Category

def get_sales_by_region_category_bubble_chart(df: pd.DataFrame) -> alt.LayerChart:
    """
    Get the bubble chart for ARPU and total sales by Region and Product Category
    Args:
        df: pd.DataFrame - Requires sales, order_count, and ARPU columns
    Returns:
        alt.Chart
    """

    bubble_chart = alt.Chart(df).mark_circle(opacity=0.7).encode(
        x=alt.X('sales:Q', title='Total Sales (BRL)'),
        y=alt.Y('ARPU:Q', title='Average Revenue per Order (ARPU)'),
        size=alt.Size('order_count:Q', title='Order Count', scale=alt.Scale(range=[30, 1000])),
        color=alt.Color('region:N', title='Region'),
        tooltip=['category_name', 'region', 'sales', 'ARPU', 'order_count']
    ).properties(
        title='Sales vs ARPU by Product Category and Region',
        width=800,
        height=500
    ).interactive()
    rule = alt.Chart(df).mark_rule(color='red', strokeWidth=2).encode(
        y=alt.Y('mean(ARPU):Q', title='Average Revenue per Order (ARPU)')
    )
    rule2 = alt.Chart(df).mark_rule(color='blue', strokeWidth=2).encode(
        x=alt.X('mean(sales):Q', title='Total Sales (BRL)')
    )

    return bubble_chart + rule + rule2

sales_by_region_category_bubble_chart = get_sales_by_region_category_bubble_chart(sales_by_region_category)

sales_by_region_category_bubble_chart


alt.LayerChart(...)

**Takeaway:** Increase marketing efforts in products with higher-than average ARPU and sales.

Examples:
- health_beauty, Northeast
- watches_gifts, Northeast
- cool_stuff, Northeast
- auto, Northeast
- watches_gifts, Central-West
- cool_stuff, South

**Next Steps:** 
- Aggregate higher than average products and minimize for find freight price.
- View over time


## View sales over time

In [27]:
def get_average_sales_ARPU(sales_by_region: pd.DataFrame, 
                                                    df_order: pd.DataFrame, 
                                                    df_order_item: pd.DataFrame, 
                                                    df_product: pd.DataFrame,
                                                    sales: bool = True,
                                                    ARPU: bool = False,
                                                    top_n: int = 10) -> pd.DataFrame:
    """
    Get the above average sales and below average ARPU
    Args:
        sales_by_region: pd.DataFrame
        df_order: pd.DataFrame
        df_order_item: pd.DataFrame
        df_product: pd.DataFrame
        sales: bool -> True if above average sales, False if below average sales
        ARPU: bool -> True if below average ARPU, False if above average ARPU
    Returns:
        pd.DataFrame - Columns: arpu, region, sales, product_category, order_purchase_month
    
    Notes:
        - The merging may be redundant and could be done in calculating sales_by_region
    """
    
    # Means
    avg_ARPU = sales_by_region["ARPU"].mean()
    avg_sales = sales_by_region['sales'].mean()

    if sales:
        sales_mask = (sales_by_region['sales'] > avg_sales)
    else:
        sales_mask = (sales_by_region['sales'] < avg_sales)
    if ARPU:
        ARPU_mask = (sales_by_region['ARPU'] < avg_ARPU)
    else:
        ARPU_mask = (sales_by_region['ARPU'] > avg_ARPU)

    mask = sales_mask & ARPU_mask

    # Top 10
    above_avg = sales_by_region.loc[mask].sort_values(by=['sales']).head(top_n)

    # Merge order, order_item, product, and above average to grab top sales
    merge = (df_order
            .merge(df_order_item)
            .merge(df_product)
            .merge(above_avg, left_on="category_name", right_on="category_name")
            [['ARPU','region','sales', "category_name", "purchase_month"]])

    merge['purchase_month'] = pd.to_datetime(merge['purchase_month'])
    return merge

above_average_sales_below_average_arpu = get_average_sales_ARPU(sales_by_region_category,
                                                                 df_order,
                                                                df_order_item, 
                                                                df_product, 
                                                                sales=True, 
                                                                ARPU=False)

below_average_sales_above_average_arpu = get_average_sales_ARPU(sales_by_region_category,
                                                                 df_order,
                                                                df_order_item, 
                                                                df_product, 
                                                                sales=False, 
                                                                ARPU=True)


In [28]:
def sales_ARPU_time_chart(df: pd.DataFrame, year: int | list[int], title: str) -> alt.Chart | None:
    """
    Above Average Sales and Below Average ARPU
    Args: 
        sales_by_region: pd.DataFrame
            Columns: Product Category, Region, Sales, Orders, ARPU
        df_order: pd.DataFrame
        df_order_item: pd.DataFrame
        df_product: pd.DataFrame
    Returns: 
        alt.Chart - Above Average Sales and Below Average ARPU
    """

    if isinstance(year, int) and year not in [2016, 2017, 2018, 2019]:
        print(f"Year {year} not in [2016, 2017, 2018, 2019]")
        return
    if isinstance(year, list):
        if not all(y in [2016, 2017, 2018, 2019] for y in year):
            print("All years must be in [2016, 2017, 2018, 2019]")
            return

    df_year = df[df['purchase_month'].dt.year.isin([year])]

    # Group by month and category
    df_year_agg = df_year.groupby(
        ['purchase_month', 'category_name']
            ).size().reset_index(name='order_count')

    chart = alt.Chart(df_year_agg).mark_line(point=True).encode(
        x=alt.X('purchase_month:T', title='Month'),
        y=alt.Y('order_count:Q', title='Number of Orders'),
        color=alt.Color('category_name:N', title='Product Category'),
        tooltip=['purchase_month:T', 'category_name:N', 'order_count:Q']
    ).properties(
        title=title,
        width=700,
        height=400
    ).interactive()

    return chart

above_average_sales_below_average_arpu_chart = sales_ARPU_time_chart(above_average_sales_below_average_arpu, 2017, title="Above Average Sales and Below Average ARPU")
below_average_sales_above_average_arpu_chart = sales_ARPU_time_chart(below_average_sales_above_average_arpu, 2017, title="Below Average Sales and Above Average ARPU")

above_average_sales_below_average_arpu_chart | below_average_sales_above_average_arpu_chart


alt.HConcatChart(...)

**Observation:** Housewares outperform other categories and have high seasonality.

**Recommendation:** Consider looking into logistical cost-saving measures to increase ARPU during spikes. 

**Observation:** Computer sales outperform the rest of the above average ARPU, with high seasonality.

**Recommendation:** Consider increasing marketing efforts in earlier months to boost sales. 

### Customer Analysis

In [ ]:

#todo Move to preprocessing
df_order['purchase_week'] = df_order['purchase_dayofyear'] // 7 % 52 + 1

In [ ]:
def find_frequent_customers(data: dict, freq: int) -> pd.DataFrame:

    df_order = data['order']

    customer_count = (
        df_order.groupby(['purchase_week', 'customer_id'])
          .size()
          .reset_index(name='purchase_count')
    )
    frequent_customers = (
        customer_count.loc[customer_count['purchase_count'] == freq, 'customer_id']
        .drop_duplicates()
        .to_frame()
    )
    
    return frequent_customers

#todo Change I/O to data dictionary for EDA
data = {'order': df_order, 'order_item': df_order_item, 'customer': df_customer}

In [ ]:
def find_products_by_customer_freq(data, top_n, freq):
    df_order_item = data['order_item']
    
    frequent_customers = find_frequent_customers(data, freq)

    order_items_customers = df_order_item.merge(df_order[['order_id', 'customer_id']], on='order_id', how='right')
    
    merged_freq = order_items_customers.merge(frequent_customers['customer_id'])

    grouped_freq = merged_freq.groupby('category_name')['order_id'].count().sort_values(ascending=False)

    return grouped_freq.head(top_n)

find_products_by_customer_freq(data, 10, 5)

category_name
furniture_decor          780
computers_accessories    490
garden_tools             435
health_beauty            390
housewares               345
bed_bath_table           315
office_furniture         300
sports_leisure           175
watches_gifts            125
home_construction        105
Name: order_id, dtype: int64

In [246]:
from typing import Literal

def find_states_by_customer_freq(data, top_n, freq, locale: Literal['state', 'city', 'region']):
    df_customer = data['customer']

    frequent_customers = find_frequent_customers(data, freq)

    top_states = (df_customer.merge(frequent_customers)
                    .groupby(locale)['customer_id']
                    .count().sort_values(ascending=False)
                    .head(top_n))
    return top_states

find_states_by_customer_freq(data, 10, 5, 'state')

state
SP    85
RJ    25
MG    23
RS    16
PR    12
BA     6
SC     4
CE     3
PI     1
SE     1
Name: customer_id, dtype: int64